In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load trained model
classifier = load_model('sign_language_model.h5')

# Image dimensions
image_x, image_y = 64, 64

labels = [
    "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L",
    "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X",
    "Y", "Z"
]
# Debugging: Display Preprocessed Image
def debug_image(display_title, img):
    cv2.imshow(display_title, img)
    cv2.waitKey(1)

# Prediction function
def predictor(image_path):
    test_image = load_img(image_path, target_size=(image_x, image_y), color_mode='grayscale')
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image / 255.0  # Normalize

    result = classifier.predict(test_image, verbose=0)
    predicted_index = np.argmax(result)
    print(f"Raw Prediction Probabilities: {result}")
    return labels[predicted_index]

# Webcam for real-time prediction
cam = cv2.VideoCapture(0)
img_text = ""

while True:
    ret, frame = cam.read()
    if not ret:
        print("Error accessing the webcam.")
        break

    frame = cv2.flip(frame, 1)

    # Region of Interest (ROI)
    x1, y1, x2, y2 = 425, 100, 625, 300
    roi = frame[y1:y2, x1:x2]

    # Preprocess ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    mask = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Debugging: Display ROI and Preprocessed Image
    debug_image("ROI", roi)
    debug_image("Processed Image", mask)

    # Save and Predict
    temp_img_path = "temp_image.png"
    processed_image = cv2.resize(mask, (image_x, image_y))
    cv2.imwrite(temp_img_path, processed_image)
    img_text = predictor(temp_img_path)

    # Display Prediction
    cv2.putText(frame, f"Prediction: {img_text}", (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0, 255, 0), 2)

    # Show Frame
    cv2.imshow("Hand Gesture Recognition", frame)

    # Exit on 'ESC'
    if cv2.waitKey(1) & 0xFF == 27:
        break

cam.release()
cv2.destroyAllWindows()


Raw Prediction Probabilities: [[6.0558878e-03 1.1823727e-04 5.1693331e-02 3.6301040e-05 6.5603366e-05
  9.0774447e-05 2.0174716e-04 3.7372878e-04 2.0498871e-06 3.9743465e-01
  1.1358943e-06 9.1706272e-05 3.0143331e-03 6.4896800e-02 3.9645229e-03
  4.6754540e-05 2.0860267e-05 8.7220082e-04 5.2989203e-06 1.9236955e-04
  1.2535766e-01 2.3032548e-01 2.6319141e-04 2.3523867e-06 1.9264207e-05
  1.1485379e-01]]
Raw Prediction Probabilities: [[2.4152035e-01 2.1255306e-05 1.2212049e-01 3.1783778e-04 4.2521138e-02
  1.1040231e-01 3.6980015e-05 9.0437590e-07 9.6311951e-06 9.7992122e-03
  5.3643616e-06 1.0571566e-05 8.5859749e-05 2.2002982e-02 5.9946772e-04
  3.6249078e-06 2.8311923e-05 3.0989322e-05 8.6685086e-06 5.5101027e-06
  4.6604435e-04 1.2732714e-01 6.1621304e-06 5.8276378e-08 2.9313054e-07
  3.2266882e-01]]
Raw Prediction Probabilities: [[1.57267004e-02 2.05437073e-06 2.88012018e-03 8.47653428e-04
  1.73264489e-01 1.65840045e-01 1.54270256e-05 5.58806823e-09
  1.56065312e-06 4.19689387e-0